In [146]:
pip install matplotlib

You should consider upgrading via the '/Users/snow/.virtualenvs/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [147]:
pip install statsmodels

You should consider upgrading via the '/Users/snow/.virtualenvs/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [148]:
pip install tqdm

You should consider upgrading via the '/Users/snow/.virtualenvs/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [149]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd
import statsmodels.api as sm
from tqdm import tqdm
from multiprocessing.dummy import Pool
import statsmodels.formula.api as smf
pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)

In [150]:
crsp_df = pd.read_csv("crsp.zip", compression='zip',header=0,
             parse_dates = ['date','DCLRDT','RCRDDT'],
             dtype={"PRC": np.float64,
                  "VOL": np.float64,
                  "SHROUT": np.float64,
                  "DIVAMT": np.float64,
                  "SPREAD": np.float64,
                  "SHRCD":str,
                  "DISTCD":str,
                  "COMNAM": str,
                  "TICKER": str,
                   'CUSIP': str})
#Data Preprocessing
crsp_df = crsp_df[(crsp_df.SHRCD.isin(('10','11')))]
crsp_df = crsp_df[(crsp_df.date >= '1927-01-01')]
crsp_df = crsp_df[(crsp_df.date <= '2011-12-31')]
cusips_DISTCD_12 = crsp_df[crsp_df.DISTCD.apply(lambda x: x[:2]=='12' if isinstance(x, str) else False)].CUSIP.unique()
crsp_df = crsp_df[crsp_df.CUSIP.isin(cusips_DISTCD_12)]

crsp_df.PRC = crsp_df.PRC.abs()


crsp_df['year'] = crsp_df['date'].dt.year
crsp_df['month'] = crsp_df['date'].dt.month
crsp_df.sort_values(by=['CUSIP','date'], ascending=True, inplace=True)
crsp_df['PRC_t-1'] = crsp_df.groupby('CUSIP')['PRC'].shift(1)
crsp_df.head()

,PERMNO,date,SHRCD,TICKER,COMNAM,CUSIP,DCLRDT,RCRDDT,DISTCD,DIVAMT,PRC,VOL,RET,SHROUT,SPREAD,RETX,year,month,PRC_t-1
1359100,25881,1970-11-30,10,AE,A & E PLASTIK PAK INC,00003210,NaT,NaT,NaN,NaN,9.000,318.0,C,2655.0,NaN,C,1970,11,NaN
1359101,25881,1970-12-31,10,AE,A & E PLASTIK PAK INC,00003210,NaT,NaT,NaN,NaN,10.000,646.0,0.111111,2655.0,NaN,0.111111,1970,12,9.000
1359102,25881,1971-01-29,10,AE,A & E PLASTIK PAK INC,00003210,NaT,NaT,NaN,NaN,9.000,636.0,-0.100000,2655.0,NaN,-0.100000,1971,1,10.000
1359103,25881,1971-02-26,10,AE,A & E PLASTIK PAK INC,00003210,NaT,NaT,NaN,NaN,11.125,2155.0,0.236111,2655.0,NaN,0.236111,1971,2,9.000
1359104,25881,1971-03-31,10,AE,A & E PLASTIK PAK INC,00003210,NaT,NaT,NaN,NaN,9.750,2062.0,-0.123596,2655.0,NaN,-0.123596,1971,3,11.125


In [151]:
crsp_div_df = crsp_df[['date','CUSIP','DCLRDT','RCRDDT','DISTCD','DIVAMT','PRC','VOL','RET','SHROUT','SPREAD','year','month']].copy()
crsp_div_df.sort_values(by=['CUSIP','date'], ascending=True, inplace=True)
crsp_div_df = crsp_div_df.groupby(by=['CUSIP','date']).agg({
     'DCLRDT': 'last',
     'RCRDDT': 'last',
     'DISTCD': 'last',
     'DIVAMT': 'sum',
     'RET':'max',
     'VOL': 'last',
     'PRC': 'last',
     'SPREAD': 'last',
     'SHROUT': 'last'})
crsp_div_df['MCAP'] = crsp_div_df['PRC'] * crsp_div_df['SHROUT']
crsp_div_df

DCLRDT RCRDDT DISTCD  DIVAMT        RET        VOL  \
CUSIP    date                                                            
00003210 1970-11-30    NaT    NaT   None     0.0          C      318.0   
         1970-12-31    NaT    NaT   None     0.0   0.111111      646.0   
         1971-01-29    NaT    NaT   None     0.0  -0.100000      636.0   
         1971-02-26    NaT    NaT   None     0.0   0.236111     2155.0   
         1971-03-31    NaT    NaT   None     0.0  -0.123596     2062.0   
...                    ...    ...    ...     ...        ...        ...   
N5946510 2011-08-31    NaT    NaT   None     0.0  -0.089113  2419635.0   
         2011-09-30    NaT    NaT   None     0.0  -0.181205  1672860.0   
         2011-10-31    NaT    NaT   None     0.0   0.151854  1538552.0   
         2011-11-30    NaT    NaT   None     0.0  -0.002044  1220142.0   
         2011-12-30    NaT    NaT   None     0.0   0.098822  1185432.0   

                        PRC  SPREAD    SHROUT         MCAP  
CUSIP    date                                               
00003210 1970-11-30   9.000     NaN    2655.0    23895.000  
         1970-12-31  10.000     NaN    2655.0    26550.000  
         1971-01-29   9.000     NaN    2655.0    23895.000  
         1971-02-26  11.125     NaN    2655.0    29536.875  
         1971-03-31   9.750     NaN    2655.0    25886.250  
...                     ...     ...       ...          ...  
N5946510 2011-08-31  20.750     NaN  426263.0  8844957.250  
         2011-09-30  16.990     NaN  426485.0  7245980.150  
         2011-10-31  19.570     NaN  426512.0  8346839.840  
         2011-11-30  19.530     NaN  426512.0  8329779.360  
         2011-12-30  21.460     NaN  426678.0  9156509.880  

[2097733 rows x 10 columns]

In [152]:
crsp_div_df.sort_values(by=['CUSIP','date'], ascending=True, inplace=True)
crsp_div_df['freq'] = None
crsp_div_df['freq'] = crsp_div_df[crsp_div_df.RCRDDT.isna()==False]['DISTCD'].apply(lambda x: x[2:3] if x is not None else None)
crsp_div_df['freq'] = crsp_div_df.groupby(by=['CUSIP']).fillna(method='ffill', limit=11)['freq']


In [153]:
crsp_div_df['div_yr']=np.where(crsp_div_df['freq']=='6',True,False )
crsp_div_df['div_semi_annual']=np.where(crsp_div_df['freq']=='4',True,False)
crsp_div_df['div_quarterly']=np.where(crsp_div_df['freq']=='3',True,False)
crsp_div_df['div_unknown']=np.where(crsp_div_df['freq']=='0',True,False )
crsp_div_df['div_monthly']=np.where(crsp_div_df['freq']=='2',True,False )
crsp_temp=crsp_div_df


In [154]:
crsp_temp['lag_u_1']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(1)
crsp_temp['lag_u_2']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(2)
crsp_temp['lag_u_3']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(3)
crsp_temp['lag_u_4']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(4)
crsp_temp['lag_u_5']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(5)
crsp_temp['lag_u_6']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(6)
crsp_temp['lag_u_7']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(7)
crsp_temp['lag_u_8']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(8)
crsp_temp['lag_u_9']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(9)
crsp_temp['lag_u_10']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(10)
crsp_temp['lag_u_11']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(11)
crsp_temp['lag_u_12']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(12)

crsp_temp['lag_quarterly']=crsp_temp.groupby('CUSIP')['div_quarterly'].shift(3)
crsp_temp['lag_semi_annual_six']=crsp_temp.groupby('CUSIP')['div_semi_annual'].shift(6)
crsp_temp['lag_semi_annual_twl']=crsp_temp.groupby('CUSIP')['div_semi_annual'].shift(12)
crsp_temp['lag_div_yr']=crsp_temp.groupby('CUSIP')['div_yr'].shift(12)
crsp_temp['lag_1']=crsp_temp.groupby('CUSIP')['div_monthly'].shift(1)
crsp_temp['lag_4']=crsp_temp.groupby('CUSIP')['div_monthly'].shift(4)
crsp_temp['lag_7']=crsp_temp.groupby('CUSIP')['div_monthly'].shift(7)
crsp_temp['lag_10']=crsp_temp.groupby('CUSIP')['div_monthly'].shift(10)

crsp_div_df['lag_month']=crsp_temp.groupby('CUSIP')['div_monthly'].shift(1)

crsp_temp

DCLRDT RCRDDT DISTCD  DIVAMT        RET        VOL  \
CUSIP    date                                                            
00003210 1970-11-30    NaT    NaT   None     0.0          C      318.0   
         1970-12-31    NaT    NaT   None     0.0   0.111111      646.0   
         1971-01-29    NaT    NaT   None     0.0  -0.100000      636.0   
         1971-02-26    NaT    NaT   None     0.0   0.236111     2155.0   
         1971-03-31    NaT    NaT   None     0.0  -0.123596     2062.0   
...                    ...    ...    ...     ...        ...        ...   
N5946510 2011-08-31    NaT    NaT   None     0.0  -0.089113  2419635.0   
         2011-09-30    NaT    NaT   None     0.0  -0.181205  1672860.0   
         2011-10-31    NaT    NaT   None     0.0   0.151854  1538552.0   
         2011-11-30    NaT    NaT   None     0.0  -0.002044  1220142.0   
         2011-12-30    NaT    NaT   None     0.0   0.098822  1185432.0   

                        PRC  SPREAD    SHROUT         MCAP  ... lag_u_12  \
CUSIP    date                                               ...            
00003210 1970-11-30   9.000     NaN    2655.0    23895.000  ...      NaN   
         1970-12-31  10.000     NaN    2655.0    26550.000  ...      NaN   
         1971-01-29   9.000     NaN    2655.0    23895.000  ...      NaN   
         1971-02-26  11.125     NaN    2655.0    29536.875  ...      NaN   
         1971-03-31   9.750     NaN    2655.0    25886.250  ...      NaN   
...                     ...     ...       ...          ...  ...      ...   
N5946510 2011-08-31  20.750     NaN  426263.0  8844957.250  ...    False   
         2011-09-30  16.990     NaN  426485.0  7245980.150  ...    False   
         2011-10-31  19.570     NaN  426512.0  8346839.840  ...    False   
         2011-11-30  19.530     NaN  426512.0  8329779.360  ...    False   
         2011-12-30  21.460     NaN  426678.0  9156509.880  ...    False   

                     lag_quarterly  lag_semi_annual_six  lag_semi_annual_twl  \
CUSIP    date                                                                  
00003210 1970-11-30            NaN                  NaN                  NaN   
         1970-12-31            NaN                  NaN                  NaN   
         1971-01-29            NaN                  NaN                  NaN   
         1971-02-26          False                  NaN                  NaN   
         1971-03-31          False                  NaN                  NaN   
...                            ...                  ...                  ...   
N5946510 2011-08-31          False                False                False   
         2011-09-30          False                False                False   
         2011-10-31          False                False                False   
         2011-11-30          False                False                False   
         2011-12-30          False                False                False   

                     lag_div_yr  lag_1  lag_4  lag_7 lag_10 lag_month  
CUSIP    date                                                          
00003210 1970-11-30         NaN    NaN    NaN    NaN    NaN       NaN  
         1970-12-31         NaN  False    NaN    NaN    NaN     False  
         1971-01-29         NaN  False    NaN    NaN    NaN     False  
         1971-02-26         NaN  False    NaN    NaN    NaN     False  
         1971-03-31         NaN  False  False    NaN    NaN     False  
...                         ...    ...    ...    ...    ...       ...  
N5946510 2011-08-31       False  False  False  False  False     False  
         2011-09-30       False  False  False  False  False     False  
         2011-10-31       False  False  False  False  False     False  
         2011-11-30       False  False  False  False  False     False  
         2011-12-30       False  False  False  False  False     False  

[2097733 rows x 37 columns]

In [155]:
# Strategies 1: Long predicted dividend and short all other companies
crsp_temp['long_stocks'] = (crsp_temp['lag_u_3'] ==True) | (crsp_temp['lag_u_6'] ==True) | ( crsp_temp['lag_u_9'] ==True ) | (crsp_temp['lag_u_12'] ==True ) | ( crsp_temp['lag_quarterly'] ==True ) | (crsp_temp['lag_semi_annual_six'] ==True ) | (crsp_temp['lag_semi_annual_twl'] ==True) |(crsp_temp['lag_div_yr'] ==True)

# crsp_temp['short_stocks'] = np.where(crsp_temp['long_stocks']==False,True,False)

# Strategies 2: Long predicted dividend and short all other dividend payers companies
# crsp_temp['short_stocks'] = (crsp_temp['lag_u_1'] ==True) | (crsp_temp['lag_u_2'] ==True) | ( crsp_temp['lag_u_4'] ==True ) | (crsp_temp['lag_u_5'] ==True ) | ( crsp_temp['lag_u_8'] ==True ) | (crsp_temp['lag_u_10'] ==True ) | (crsp_temp['lag_u_11'] ==True )| (crsp_div_df['lag_month']==True)

# Strategies 3: Long predicted dividend and short all months occurring 1,4,7 or 10 months ago
crsp_temp['short_stocks'] = (crsp_temp['lag_1'] ==True) | (crsp_temp['lag_4'] ==True) | ( crsp_temp['lag_7'] ==True ) | (crsp_temp['lag_10'] ==True ) 
crsp_temp['short_stocks'].describe()



count     2097733
unique          2
top         False
freq      1969796
Name: short_stocks, dtype: object

In [156]:
def long_weights(group):
    # Select all stocks with long position from the group
    longs = group.loc[group['long_stocks'] == True]
    # Get market capitalization of each long stock
    mktcap = longs['SHROUT'] * longs['PRC']
    # Calculate long weights by market captalization
    longweights = mktcap / mktcap.sum()
    # Assign weights for the group of long stocks
    group.loc[group['long_stocks']==True, 'long_weight'] = longweights
    group['long_weight'] = group['long_weight'].shift(1)
    return group

def short_weights(group):
    # Select all stocks with short position from the group
    shorts = group.loc[group['short_stocks'] == True]
    # Get market capitalization of each long stock
    mktcap = shorts['SHROUT'] * shorts['PRC']
    # Calculate long weights by market captalization
    shortweights = mktcap / mktcap.sum()
    # Assign weights for the group of long stocks
    group.loc[group['short_stocks']==True, 'short_weight'] = shortweights
    return group

def cal_return_vw(group):
    group=group[(group.RET<='65')]
    group['port_return_long_vw'] = pd.to_numeric(group['RET']) * pd.to_numeric(group['long_weight'])
    longport = pd.pivot_table(group, values='port_return_long_vw', index='date', aggfunc=np.sum)

    group['port_return_short_vw'] = pd.to_numeric(group['RET']) * pd.to_numeric(group['short_weight'])
    shortport = pd.pivot_table(group, values='port_return_short_vw', index='date', aggfunc=np.sum)

    data=pd.merge(longport, shortport, on='date')
    # print(data)
    data['longshort_eq'] = data['port_return_long_vw'] - data['port_return_short_vw']
    return longport,shortport,data

def cal_return_eq(group):
    
    longs = group.loc[group['long_stocks'] == True]

    shorts = group.loc[group['short_stocks'] == True]
    
    longs=longs[(longs.RET<='65')]
    shorts=shorts[(shorts.RET<='65')]
    longs['port_return_long_eq'] = pd.to_numeric(longs['RET']).mean()
    longport = pd.pivot_table(longs, values='port_return_long_eq', index='date', aggfunc=np.sum)

    shorts['port_return_short_eq'] = pd.to_numeric(shorts['RET']).mean()
    shortport = pd.pivot_table(shorts, values='port_return_short_eq', index='date', aggfunc=np.sum)

    data=pd.merge(longport, shortport, on='date')
    # print(data)
    data['longshort_eq'] = data['port_return_long_eq'] - data['port_return_short_eq']
    print(data)
    # print(shortport)
    # print(longshortport)
    
    return longport,shortport,data

panel=long_weights(crsp_temp)
panel=short_weights(panel)
longport_vw,shortport_vw,longshortport_vw=cal_return_vw(panel)
longport_eq,shortport_eq,longshortport_eq=cal_return_eq(crsp_temp)



<ipython-input-156-3e615cdcae9c>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['port_return_long_vw'] = pd.to_numeric(group['RET']) * pd.to_numeric(group['long_weight'])
<ipython-input-156-3e615cdcae9c>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['port_return_short_vw'] = pd.to_numeric(group['RET']) * pd.to_numeric(group['short_weight'])
            port_return_long_eq  port_return_short_eq  longshort_eq
date                                                               
1927-04-30    

In [157]:
ff4=pd.read_csv('four_factor_monthly.csv',index_col='date')
ff4.index = pd.to_datetime(ff4.index)

In [163]:
# equal_weighted_long
# data=pd.merge(ff4, longport_eq, on='date')

# (
#     smf.ols('port_return_long_eq ~ 1 + hml + mkt_rf + smb + Mom', data=pd.merge(ff4, longport_eq, on='date'))
#     .fit().summary()
# )

# equal_weighted_short
# (
#     smf.ols('port_return_short_eq ~ 1 + hml + mkt_rf + smb + Mom', data=pd.merge(ff4, shortport_eq, on='date'))
#     .fit().summary()
# )
# equal_weighted_diff
(
    smf.ols('longshort_eq ~ 1 + hml + mkt_rf + smb + Mom', data=pd.merge(ff4, longshortport_eq, on='date'))
    .fit().summary()
)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           longshort_eq   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.5352
Date:                Sat, 29 May 2021   Prob (F-statistic):              0.710
Time:                        01:36:45   Log-Likelihood:                -2770.7
No. Observations:                 744   AIC:                             5551.
Df Residuals:                     739   BIC:                             5574.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     14.0827      0.378     37.218      0.000      13.340      14.826
hml           12.3086     11.305      1.089      0.277      -9.885      34.502
mkt_rf        -1.5785      7.630     -0.207      0.836     -16.558      13.401
smb            6.4775     12.047      0.538      0.591     -17.174      30.129
Mom            8.5506      8.546      1.001      0.317      -8.226      25.328
==============================================================================
Omnibus:                      492.702   Durbin-Watson:                   0.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               48.795
Skew:                           0.195   Prob(JB):                     2.54e-11
Kurtosis:                       1.808   Cond. No.                         33.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [165]:
# value_weighted_long
data=pd.merge(ff4, longport_eq, on='date')

# (
#     smf.ols('port_return_long_vw ~ 1 + hml + mkt_rf + smb + Mom', data=pd.merge(ff4, longport_vw, on='date'))
#     .fit().summary()
# )

# value_weighted_short
# (
#     smf.ols('port_return_short_vw ~ 1 + hml + mkt_rf + smb + Mom', data=pd.merge(ff4, shortport_vw, on='date'))
#     .fit().summary()
# )
# value_weighted_diff
(
    smf.ols('longshort_eq ~ 1 + hml + mkt_rf + smb + Mom', data=pd.merge(ff4, longshortport_vw, on='date'))
    .fit().summary()
)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           longshort_eq   R-squared:                       0.122
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     25.85
Date:                Sat, 29 May 2021   Prob (F-statistic):           4.40e-20
Time:                        01:37:39   Log-Likelihood:                 5405.2
No. Observations:                 747   AIC:                        -1.080e+04
Df Residuals:                     742   BIC:                        -1.078e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.701e-05   6.57e-06     -2.590      0.010   -2.99e-05   -4.12e-06
hml            0.0015      0.000      7.810      0.000       0.001       0.002
mkt_rf        -0.0005      0.000     -4.145      0.000      -0.001      -0.000
smb           -0.0006      0.000     -3.050      0.002      -0.001      -0.000
Mom           -0.0005      0.000     -3.366      0.001      -0.001      -0.000
==============================================================================
Omnibus:                      788.319   Durbin-Watson:                   2.076
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            78417.063
Skew:                          -4.697   Prob(JB):                         0.00
Kurtosis:                      52.307   Cond. No.                         34.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [160]:
# panel B do one strategies 1 & 2 
# CAPM Alpha
(
    smf.ols('port_return_long_eq ~ 1 + mkt_rf', data=pd.merge(ff4, longport_eq, on='date'))
    .fit().summary()
)



<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     port_return_long_eq   R-squared:                       0.000
Model:                             OLS   Adj. R-squared:                 -0.001
Method:                  Least Squares   F-statistic:                   0.04730
Date:                 Sat, 29 May 2021   Prob (F-statistic):              0.828
Time:                         01:24:25   Log-Likelihood:                -2844.0
No. Observations:                  744   AIC:                             5692.
Df Residuals:                      742   BIC:                             5701.
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     15.6335      0.408     38.324      0.000      14.833      16.434
mkt_rf        -1.6002      7.358     -0.217      0.828     -16.045      12.845
==============================================================================
Omnibus:                     4247.691   Durbin-Watson:                   0.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               59.673
Skew:                           0.156   Prob(JB):                     1.10e-13
Kurtosis:                       1.648   Cond. No.                         18.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [161]:
# 3-factor Alpha
(
    smf.ols('port_return_short_eq ~ 1 + hml + mkt_rf + smb', data=pd.merge(ff4, shortport_eq, on='date'))
    .fit().summary()
)

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     port_return_short_eq   R-squared:                       0.003
Model:                              OLS   Adj. R-squared:                 -0.001
Method:                   Least Squares   F-statistic:                    0.7856
Date:                  Sat, 29 May 2021   Prob (F-statistic):              0.502
Time:                          01:24:25   Log-Likelihood:                -1279.6
No. Observations:                   746   AIC:                             2567.
Df Residuals:                       742   BIC:                             2586.
Df Model:                             3                                         
Covariance Type:              nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4448      0.050     29.048      0.000       1.347       1.542
hml            1.1131      1.482      0.751      0.453      -1.796       4.022
mkt_rf        -0.5628      0.986     -0.571      0.568      -2.499       1.373
smb           -1.6585      1.614     -1.027      0.305      -4.828       1.511
==============================================================================
Omnibus:                      109.431   Durbin-Watson:                   0.029
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              156.886
Skew:                           1.102   Prob(JB):                     8.56e-35
Kurtosis:                       3.440   Cond. No.                         33.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""